# Generate pressure coefficients

Initialize Path Manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CpPathManager
import pathlib

path_manager = CpPathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Cp config file

In [2]:
from cfdmod.use_cases.pressure.cp_config import CpCaseConfig

cfg_path = pathlib.Path("./fixtures/tests/pressure/cp_params.yaml")
post_proc_cfg = CpCaseConfig.from_file(cfg_path)

post_proc_cfg

CpCaseConfig(pressure_coefficient={'default': CpConfig(number_of_chunks=10, timestep_range=(10000.0, 20000.0), reference_pressure='average', U_H=0.05, U_H_correction_factor=1, statistics=[BasicStatisticModel(stats='mean'), BasicStatisticModel(stats='rms'), BasicStatisticModel(stats='skewness'), BasicStatisticModel(stats='kurtosis'), ParameterizedStatisticModel(stats='mean_eq', params=MeanEquivalentParamsModel(scale_factor=0.61)), ParameterizedStatisticModel(stats='min', params=ExtremeAbsoluteParamsModel(method_type='Absolute')), ParameterizedStatisticModel(stats='max', params=ExtremeGumbelParamsModel(method_type='Gumbel', peak_duration=3.0, event_duration=600.0, n_subdivisions=10, non_exceedance_probability=0.9))])}, time_scale_conversion=TimeScaleParameters(CST_real=0.56, CST_sim=16.89))

Read LNAS Mesh

In [3]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

((1549, 3), (2915, 3))

Read hist series

In [4]:
import pandas as pd

static_data_path = pathlib.Path(
    "./fixtures/tests/pressure/data/new.points.static_pressure.data.resampled.h5"
)
body_data_path = pathlib.Path("./fixtures/tests/pressure/data/new.bodies.galpao.data.resampled.h5")

Process pressure coefficient use case

In [5]:
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.cp_data import process_cp
from cfdmod.use_cases.pressure.path_manager import copy_input_artifacts

for cfg_lbl, cfg in post_proc_cfg.pressure_coefficient.items():
    logger.info(f"Processing pressure coefficients for config {cfg_lbl} ...")
    process_cp(
        pressure_data_path=static_data_path,
        body_data_path=body_data_path,
        cfg_label=cfg_lbl,
        cfg=cfg,
        mesh=mesh.geometry,
        path_manager=path_manager,
        time_scale_factor=post_proc_cfg.time_scale_conversion.time_scale,
    )

    logger.info("Copying input artifacts")
    copy_input_artifacts(
        cfg_path=cfg_path,
        mesh_path=mesh_path,
        static_data_path=static_data_path,
        body_data_path=body_data_path,
        path_manager=path_manager,
    )

    logger.info(f"Processed pressure coefficients for config {cfg_lbl}!")

[2024-05-15 16:04:58,295] [INFO] - cfdmod - Processing pressure coefficients for config default ... (113073770.py:6)
[2024-05-15 16:04:58,297] [INFO] - cfdmod - Transforming into pressure coefficient (cp_data.py:181)


a:\Aerosim\Repositories\cfdmod\cfdmod\use_cases\pressure\cp_data.py:175: RuntimeWarning: Path for time series already exists output\pressure\cp\default\time_series.h5. Deleted old file
  warnings.warn(


[2024-05-15 16:05:05,332] [INFO] - cfdmod - Dividing into point groups (cp_data.py:198)


a:\Aerosim\Repositories\cfdmod\cfdmod\use_cases\pressure\cp_data.py:192: RuntimeWarning: Path for grouped time series already exists output\pressure\cp\default\time_series.grouped.h5. Deleted old file
  warnings.warn(


[2024-05-15 16:05:09,757] [INFO] - cfdmod - Calculating statistics (cp_data.py:205)
1548


TypeError: InsertTuple1 argument 2: must be real number, not str